In [1]:
import pandas as pd
import glob
import os
import dask.dataframe as dd
import polars as pl
import re
import geopandas as gpd
import matplotlib.pyplot as plt

In [2]:
files = glob.glob('processed/city_sec_wage/*.csv')

wage = pd.DataFrame()

for file in files:
    year = int(os.path.basename(file)[-8:-4])
    current = pd.read_csv(file)
    current['city_total'] = current.iloc[:, 1:].sum(axis=1)
    current = current.melt(id_vars=['MET2013'], var_name='Sector', value_name='Wage')
    current['Year'] = year
    wage = pd.concat([wage, current], ignore_index=True)

wage.to_csv('aggregate/city_sector_wage.csv', index=False)

In [3]:
files = glob.glob('processed/sec_occ_wage/*.csv')

sec_wage = pd.DataFrame()

for file in files:
    year = int(os.path.basename(file)[-8:-4])
    current = pd.read_csv(file)
    current['sector_total'] = current.iloc[:, 1:].sum(axis=1)
    current = current.melt(id_vars=['INDNAICS'], var_name='Occupation', value_name='Wage')
    current['Year'] = year
    sec_wage = pd.concat([sec_wage, current], ignore_index=True)

sec_wage.to_csv('aggregate/sector_occupation_wage.csv', index=False)

In [ ]:
df = dd.read_csv('raw/data_new.csv', assume_missing=True, dtype={'INDNAICS': 'string'})
df.head()

In [ ]:
df = dd.read_csv('raw/usa_00010.csv', assume_missing=True, dtype={'INDNAICS': 'string'})
df = df[['MET2013', 'METAREA']]
df = df.drop_duplicates(subset=['MET2013', 'METAREA'])
df = df.compute()
df = df.dropna()

In [ ]:
df = pl.scan_csv('raw/usa_00015.csv')
df = df.filter(pl.col('YEAR') == 2019).collect()

# df = df.with_columns(
#     pl.col('OCC1990').map_elements(map_occupations, return_dtype=pl.Utf8).alias('occupation')
# )

# years = df.filter(pl.col('YEAR').is_not_null()).select(pl.col('YEAR').unique()).collect().to_series()
# current = df.filter(pl.col('YEAR') == year).collect().to_pandas()

In [ ]:
mappings = [
    ((405, 408), 'occ3_clean'),
    ((415, 427), 'occ3_protect'),
    ((415, 415), 'occ3_guard'),
    ((425, 427), 'occ3_guard'),
    ((433, 444), 'occ3_food'),
    ((445, 447), 'occ3_shealth'),
    ((448, 455), 'occ3_janitor'),
    ((457, 458), 'occ3_beauty'),
    ((459, 467), 'occ3_recreation'),
    ((468, 468), 'occ3_child'),
    ((469, 472), 'occ3_others'),
    ((3, 22), 'occ2_exec'),
    ((23, 37), 'occ2_mgmtrel'),
    ((43, 200), 'occ2_prof'),
    ((203, 235), 'occ2_tech'),
    ((243, 258), 'occ2_finsales'),
    ((274, 283), 'occ2_retsales'),
    ((303, 389), 'occ2_cleric'),
    ((417, 423), 'occ2_firepol'),
    ((473, 475), 'occ2_farmer'),
    ((479, 498), 'occ2_otheragr'),
    ((503, 549), 'occ2_mechanic'),
    ((558, 599), 'occ2_constr'),
    ((614, 617), 'occ2_mining'),
    ((628, 699), 'occ2_product'),
    ((703, 799), 'occ2_operator'),
    ((803, 889), 'occ2_transp'),
]

def map_occupations(occ1990dd):
    for (start, end), occupation in mappings:
        if start <= occ1990dd <= end:
            return occupation
    return 'other'

def expand_ranges(df):
    expanded_rows = []
    for index, row in df.iterrows():
        if '-' in index:
            start, end = map(int, index.split('-'))
            for i in range(start, end + 1):
                expanded_rows.append((str(i), row['Name']))
        else:
            expanded_rows.append((index, row['Name']))
    return pd.DataFrame(expanded_rows, columns=['Sector', 'Name']).set_index('Sector')

print('Reading data...')
df = pl.scan_csv('raw/usa_00015.csv')
met_codes = pd.read_csv('raw/met_codes.csv')
met_codes.set_index('code', inplace=True)

print('Creating occupation mappings...')
df = df.with_columns(
    pl.col('OCC1990').map_elements(map_occupations, return_dtype=pl.Utf8).alias('occupation')
)

print('Creating area mappings...')
df_cz = pd.read_csv('raw/cz_mappings.csv')
df_cz = df_cz[['LMA/CZ', 'FIPS']]
df = df.with_columns(
    (pl.col('STATEFIP').cast(pl.Int32) * 1000 + pl.col('COUNTYFIP').cast(pl.Int32)).alias('FIPS')
)

print('Mapping sectors...')
df = df.with_columns(
    pl.col('INDNAICS').cast(pl.Utf8).str.slice(0, 2).alias('INDNAICS')
)
naics_codes = pd.read_csv('raw/2022_NAICS_codes.csv')
naics_codes = naics_codes[['Sector', 'Name']]
naics_codes.dropna(inplace=True)
naics_codes.set_index('Sector', inplace=True)
naics_codes.loc['0'] = 'N/A'
naics_codes.loc['99'] = 'Unemployed'
naics_codes.loc['50'] = 'Transportation and Warehousing'
naics_codes.loc['3M'] = 'Manufacturing'
naics_codes = expand_ranges(naics_codes)

years = df.filter(pl.col('YEAR').is_not_null()).select(pl.col('YEAR').unique()).collect().to_series()

area = 'COMZONE'

current = df.filter(pl.col('YEAR') == 1980).collect()

current = current.with_columns(
    pl.col('INCWAGE').cast(pl.Float64),
    pl.col('RENT').cast(pl.Float64),
    pl.col('HHWT').cast(pl.Float64),
    pl.col('INDNAICS').cast(pl.Utf8),
)

current = current.with_columns(
    (pl.col('INCWAGE') * pl.col('HHWT')).alias('INCWAGE'),
    (pl.col('INCWAGE') / pl.col('HHWT')).alias('AVERAGE INCWAGE'),
    (pl.col('RENT') * pl.col('HHWT')).alias('RENT'),
)

# current['INCWAGE'] = current['INCWAGE'] * current['HHWT']
# current['AVERAGE INCWAGE'] = current['INCWAGE'] / current['HHWT']
# current['RENT'] = current['RENT'] * current['HHWT']

current = current.with_columns(
    (pl.col('RENT').drop_nulls().cast(pl.Int32) * pl.col('HHWT')).alias('HH_RENT')
)

# current['HH_RENT'] = current['RENT'].notnull().astype(int)
# current['HH_RENT'] = current['HH_RENT'] * current['HHWT']

naics_keys = naics_codes.index.tolist()
naics_values = naics_codes['Name'].tolist()

fips_keys = df_cz.set_index('FIPS')['LMA/CZ'].index.tolist()
fips_values = df_cz.set_index('FIPS')['LMA/CZ'].tolist()

current = current.with_columns(
    pl.col('INDNAICS').replace(naics_keys, naics_values),
    pl.col('FIPS').replace(fips_keys, fips_values).alias('COMZONE')
)

# current['INDNAICS'] = current['INDNAICS'].map(naics_codes['Name'])
# current['COMZONE'] = current['FIPS'].map(df_cz.set_index('FIPS')['LMA/CZ'])

city_occ = current.pivot(index=area, on='occupation', values='HHWT', aggregate_function='sum')
city_occ_wage = current.pivot(index=area, on='occupation', values='AVERAGE INCWAGE', aggregate_function='mean')
city_sector_wage = current.pivot(index=area, on='INDNAICS', values='INCWAGE', aggregate_function='sum')
sector_occ_wage = current.pivot(index=area, on='occupation', values='INCWAGE', aggregate_function='sum')

city_rent = current.pivot(index=area, on='RENT', aggregate_function='sum')

Reading data...
Creating occupation mappings...
Creating area mappings...
Mapping sectors...


In [ ]:
df_cw = pd.read_stata('raw/cw_ctygrp1980_czone_corr.dta')
df_cw = df_cw.rename(columns={'ctygrp1980': 'puma1980'})
df = df.merge(df_cw, on='puma1980', how='left')

In [ ]:
city_occ = df.pivot_table(index='czone', columns='OCC1990', values='HHWT', aggfunc='sum')

In [7]:
df_cw = pd.read_csv('raw/cw_puma2000_czone.csv', encoding='latin1')
# df_names = pd.read_csv('raw/puma_names.csv')
# df_names['puma2000'] = df_names['State10'] * 10000 + df_names['PUMA10']
# df_names = df_names[['puma2000', 'PUMA10_Name']]
df_names = pd.read_csv('raw/cz_names.csv')
df_cw = df_cw.merge(df_names, on='czone', how='left')
df_cw = df_cw.sort_values(by=['czone', 'afactor'])
df_cw = df_cw.drop_duplicates(subset=['czone'])
df_cw = df_cw[['czone', 'County Name']]
df_cw = df_cw.rename(columns={'czone': 'COMZONE', 'County Name': 'NAME'})

def get_data(directory, field_name):
    files = glob.glob(directory)
    files = [f for f in files if re.search(r'_(1980|199[0-9]|20[0-9]{2})\.csv$', f)]

    data = pd.DataFrame()

    for file in files:
        year = int(os.path.basename(file)[-8:-4])
        current = pd.read_csv(file)
        # current['city_total'] = current.iloc[:, 1:].sum(axis=1)
        current = current.melt(id_vars=['COMZONE'], var_name='Occupation', value_name=field_name)
        current['Year'] = year
        data = pd.concat([data, current], ignore_index=True)

    return data

employment = get_data('processed/city_occ_employment/*.csv', 'Employed')
employment = employment.merge(df_cw, on='COMZONE', how='left')
employment = employment[['Year', 'COMZONE', 'NAME', 'Occupation', 'Employed']]

# wage = get_data('processed/city_occ_wage/*.csv', 'Wage')
wb = get_data('processed/city_occ_wb/*.csv', 'Wage_Bill')

# final = employment.merge(wage, on=['Year', 'COMZONE', 'Occupation'], how='left')
final = employment.merge(wb, on=['Year', 'COMZONE', 'Occupation'], how='left')
# final.to_csv('master.csv', index=False)

df_cpi = pd.read_csv('raw/CPI.csv')
df_cpi['DATE'] = pd.to_datetime(df_cpi['DATE'])
df_cpi['Year'] = df_cpi['DATE'].dt.year
df_cpi['month'] = df_cpi['DATE'].dt.month
df_cpi = df_cpi[df_cpi['month'] == 12]
df_cpi = df_cpi.rename(columns={'CPIAUCSL': 'CPI'})
df_cpi = df_cpi[['Year', 'CPI']]

base_year = 1990

final = final.merge(df_cpi, on='Year', how='left')
final['Wage'] = final['Wage_Bill'] / final['Employed']
final['Wage_Bill'] = (final['Wage_Bill'] / final['CPI']) * df_cpi[df_cpi['Year'] == base_year]['CPI'].values[0]
final = final[['Year', 'COMZONE', 'NAME', 'Occupation', 'Employed', 'Wage', 'Wage_Bill', 'CPI']]

total_years = final['Year'].nunique()
occupation_counts = final.groupby('Occupation')['Year'].nunique()
valid_occupations = occupation_counts[occupation_counts == total_years].index

final = final[final['Occupation'].isin(valid_occupations)]

occupations = final['Occupation'].unique().tolist()
cities = final['COMZONE'].unique().tolist()
years = final['Year'].unique().tolist()
occupation_indices = {occupation: idx for idx, occupation in enumerate(occupations)}
city_indices = {city: idx for idx, city in enumerate(cities)}
year_indices = {year: idx for idx, year in enumerate(years)}

final['c'] = final['COMZONE'].map(city_indices)
final['k'] = final['Occupation'].map(occupation_indices)
final['t'] = final['Year'].map(year_indices)
final['ckt'] = final['c'].astype(str) + final['k'].astype(str) + final['t'].astype(str)

# final.to_csv('master.csv', index=False)

# final = pd.read_csv('master.csv')

years = [1990, 2000, 2009, 2018]
final = final[final['Year'].isin(years)]
final.to_csv('master_subset.csv', index=False)

occupation_df = pd.DataFrame(list(occupation_indices.items()), columns=['Occupation', 'Index'])
city_df = pd.DataFrame(list(city_indices.items()), columns=['City', 'Index'])
year_df = pd.DataFrame(list(year_indices.items()), columns=['Year', 'Index'])
# occupation_df.to_csv('occupation_indices.csv', index=False)
# city_df.to_csv('city_indices.csv', index=False)
# year_df.to_csv('year_indices.csv', index=False)


/tmp/ipykernel_458393/283644184.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data = pd.concat([data, current], ignore_index=True)
/tmp/ipykernel_458393/283644184.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data = pd.concat([data, current], ignore_index=True)


In [ ]:

df_names_cw = pd.read_csv('raw/puma_fips.csv')
df_names = pd.read_csv('raw/cz_mappings.csv')
df_pop = pd.read_csv('raw/archive/cz_county.csv')
df_puma_cz = pd.read_csv('raw/cw_puma2000_czone.csv', encoding='latin1')

df_names_cw['puma2000'] = df_names_cw['STATEFP'] * 10000 + df_names_cw['PUMA5CE']
df_names_cw['FIPS'] = df_names_cw['STATEFP'] * 1000 + df_names_cw['COUNTYFP']
df_names_cw = df_names_cw.drop_duplicates(subset=['puma2000', 'FIPS'])
df_names_cw = df_names_cw[['puma2000', 'FIPS']]

df_names = df_names[['FIPS', 'County Name']]
df_names_cw = df_names_cw.merge(df_names, on='FIPS', how='left')

df_names_cw = df_names_cw.merge(df_pop[['FIPS', 'Labor Force']], on='FIPS', how='left')

df_names_cw = df_names_cw.merge(df_puma_cz, on='puma2000', how='left')

idx = df_names_cw.groupby(['czone'])['Labor Force'].idxmax()
df_names_cw = df_names_cw.loc[idx].reset_index(drop=True)
df_names_cw = df_names_cw[['czone', 'County Name']]

# df_names_cw.to_csv('raw/cz_names.csv', index=False)

In [ ]:
def expand_ranges(df):
    expanded_rows = []
    for index, row in df.iterrows():
        if '-' in index:
            start, end = map(int, index.split('-'))
            for i in range(start, end + 1):
                expanded_rows.append((str(i), row['Name']))
        else:
            expanded_rows.append((index, row['Name']))
    return pd.DataFrame(expanded_rows, columns=['Sector', 'Name']).set_index('Sector')

naics_codes = pd.read_csv('raw/2022_NAICS_codes.csv')
naics_codes = naics_codes[['Sector', 'Name']]
naics_codes.dropna(inplace=True)
naics_codes.set_index('Sector', inplace=True)
naics_codes.loc['0'] = 'N/A'
naics_codes.loc['99'] = 'Unemployed'
naics_codes.loc['50'] = 'Transportation and Warehousing'
naics_codes.loc['3M'] = 'Manufacturing'
naics_codes = expand_ranges(naics_codes)

sic_to_naics = pd.read_csv('raw/full_sic87_naics97.csv')
sic_to_naics = sic_to_naics[['sic87', 'naics97']]
sic_to_naics = sic_to_naics.drop_duplicates(subset=['sic87'])

city_sec = pd.read_csv('raw/efsy_cbp_1980.csv')
city_sec['naics'] = city_sec['naics'].map(sic_to_naics.set_index('sic87')['naics97'])
city_sec['naics'] = city_sec['naics'].str[:2].replace('--', '0').astype(str)
city_sec['fips'] = city_sec['fipstate'] * 1000 + city_sec['fipscty']
city_sec = city_sec.groupby(['naics', 'fips']).agg({
    'ub': 'sum',
    'lb': 'sum',
    'fipstate': 'first',
    'fipscty': 'first',
}).reset_index()
city_sec['employment'] = (city_sec['ub'] + city_sec['lb']) / 2
city_sec['naics'] = city_sec['naics'].map(naics_codes['Name'])
# city_sec = city_sec.drop_duplicates(subset=['fips', 'naics'])
# city_sec = city_sec.pivot(index='fips', columns='naics', values='employment')

df_names_cw = pd.read_csv('raw/puma_fips.csv')
df_names = pd.read_csv('raw/cz_mappings.csv')
df_puma_cz = pd.read_csv('raw/cw_puma2000_czone.csv', encoding='latin1')
df_cz_names = pd.read_csv('raw/cz_names.csv')

df_names_cw['puma2000'] = df_names_cw['STATEFP'] * 10000 + df_names_cw['PUMA5CE']
df_names_cw['FIPS'] = df_names_cw['STATEFP'] * 1000 + df_names_cw['COUNTYFP']
df_names_cw = df_names_cw.drop_duplicates(subset=['puma2000', 'FIPS'])
df_names_cw = df_names_cw[['puma2000', 'FIPS']]

df_names = df_names[['FIPS', 'County Name']]
df_names_cw = df_names_cw.merge(df_names, on='FIPS', how='left')

df_names_cw = df_names_cw.merge(df_puma_cz, on='puma2000', how='left')
df_names_cw = df_names_cw[['FIPS', 'czone']]
df_names_cw = df_names_cw.drop_duplicates(subset=['FIPS'])

city_sec = city_sec.merge(df_names_cw, left_on='fips', right_on='FIPS', how='left')
city_sec['czone_name'] = city_sec['czone'].map(df_cz_names.set_index('czone')['County Name'])
city_sec = city_sec.drop_duplicates(subset=['czone', 'naics'])
city_sec = city_sec[['czone', 'czone_name', 'naics', 'employment']]
city_sec.to_csv('processed/city_sec_employment_1980.csv', index=False)
# city_sec = city_sec.pivot(index='czone', columns='naics', values='employment')

In [ ]:
df_cpi = pd.read_csv('raw/CPI.csv')
df_cpi['DATE'] = pd.to_datetime(df_cpi['DATE'])
df_cpi['year'] = df_cpi['DATE'].dt.year
df_cpi['month'] = df_cpi['DATE'].dt.month
df_cpi = df_cpi[df_cpi['month'] == 12]
df_cpi = df_cpi.rename(columns={'CPIAUCSL': 'CPI'})
df_cpi = df_cpi[['year', 'CPI']]


In [2]:
def get_data(directory, field_name, id_vars=['COMZONE'], var_name='Occupation'):
    files = glob.glob(directory)
    files = [f for f in files if re.search(r'_(1980|199[0-9]|20[0-9]{2})\.csv$', f)]

    data = pd.DataFrame()

    for file in files:
        year = int(os.path.basename(file)[-8:-4])
        current = pd.read_csv(file)
        current['city_total'] = current.iloc[:, 1:].sum(axis=1)
        current = current.melt(id_vars=id_vars, var_name=var_name, value_name=field_name)
        current['Year'] = year
        data = pd.concat([data, current], ignore_index=True)

    return data

city_sec_employment = get_data('processed/city_sec_employment/*.csv', 'Employed', id_vars=['COMZONE'], var_name='Sector')
sector_occ_wage = get_data('processed/sec_occ_wb/*.csv', 'Wage', id_vars=['INDNAICS'], var_name='Occupation')

years = [1990, 2000, 2010, 2018]
city_sec_employment = city_sec_employment[city_sec_employment['Year'].isin(years)]
sector_occ_wage = sector_occ_wage[sector_occ_wage['Year'].isin(years)]

city_sec_employment.to_csv('city_sector_employment_subset.csv', index=False)
sector_occ_wage.to_csv('sector_occ_wb_subset.csv', index=False)

/tmp/ipykernel_46983/1870073691.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data = pd.concat([data, current], ignore_index=True)


In [11]:
ddx_1990 = pd.read_stata('raw/crosswalk/cw_ind1990_ind1990ddx.dta')
sic_1990 = pd.read_stata('raw/crosswalk/cw_sic87_ind1990ddx.dta')
sic_naics = pd.read_stata('raw/crosswalk/cw_n97_s87.dta')

sic_1990 = sic_1990.merge(ddx_1990, on='ind1990ddx', how='outer')
naics_1990 = sic_naics.merge(sic_1990, left_on='sic4', right_on='sic87', how='inner')
naics_1990 = naics_1990[['naics6', 'ind1990']]
naics_1990 = naics_1990.drop_duplicates(subset=['naics6', 'ind1990'])

naics_1990.to_csv('raw/crosswalk/naics_1990.csv', index=False)

In [6]:
def expand_ranges(df):
    expanded_rows = []
    for index, row in df.iterrows():
        if '-' in index:
            start, end = map(int, index.split('-'))
            for i in range(start, end + 1):
                expanded_rows.append((str(i), row['Name']))
        else:
            expanded_rows.append((index, row['Name']))
    return pd.DataFrame(expanded_rows, columns=['Sector', 'Name']).set_index('Sector')

df = pl.scan_csv('raw/usa_00020.csv')
df = df.with_columns(
    (pl.col('STATEFIP').cast(pl.Int32) * 10000 + pl.col('PUMA').cast(pl.Int32)).alias('FIPS'),
    (pl.col('STATEFIP').cast(pl.Int32) * 1000 + pl.col('CNTYGP98').cast(pl.Int32)).alias('ctygrp1980'),
)
df = df.filter(pl.col('YEAR') == 1990).collect().to_pandas()

df_cz_1990 = pd.read_csv('raw/cw_puma1990_czone.csv', encoding='latin1')
df = df.merge(df_cz_1990, left_on='FIPS', right_on='puma1990', how='left')
df['COMZONE'] = df['czone']

df_cz = pd.read_csv('raw/crosswalk/naics_1990.csv')
df_cz['naics2'] = df_cz['naics6'].astype(str).str[:2]
df_cz = df_cz[['naics2', 'ind1990']]
df_cz = df_cz.drop_duplicates(subset=['naics2', 'ind1990'])

df = df.merge(df_cz, left_on='IND1990', right_on='ind1990', how='left')

naics_codes = pd.read_csv('raw/2022_NAICS_codes.csv')
naics_codes = naics_codes[['Sector', 'Name']]
naics_codes.dropna(inplace=True)
naics_codes.set_index('Sector', inplace=True)
naics_codes.loc['0'] = 'N/A'
naics_codes.loc['99'] = 'Unemployed'
naics_codes.loc['50'] = 'Transportation and Warehousing'
naics_codes.loc['3M'] = 'Manufacturing'
naics_codes = expand_ranges(naics_codes)

df['naics2'] = df['naics2'].map(naics_codes['Name'])